# The "Hello, world!" of Genetic Algorithms (GAs)


You can read the full introduction to this project on my blog at: udara.io/something. I'd suggest reading it if you're completely new to GAs and have no background in computer science. Besides that I've tried my best to keep the terminology inline with the code. 


## Background

> These early pioneers of computer science were as much interested in biology and psychology as in electronics, and they looked to natural systems as guiding metaphors for how to achieve their visions. These biologically motivated computing activities have waxed and waned over the years, but since the early 1980s they have all undergone a resurgence in the computation research community. The first has grown into the field of neural networks, the second into machine learning, and the third into what is now called "evolutionary computation," of which genetic algorithms are the most prominent example.

Many computational problems could use a bout of parallelism. Imagine you're searching for equations that can predict trends in the stock market. An algorithm that can effectively do this could benifit from some parallelism; or in other words, an ability to explore the problem space more widely.

Instead of trying to directly write and optimize one equation, imagine being able to consider a lot of possible equations at the same time. Tweaking and combining them with the hope of finding the most optimal, or a set of most optimal equations, that can predict the ups and downs of the stock market. This can (theoritically) be done with genetic algorithms; **something that can both parallelize the search and effectively select a group of solutions to evaluate**. 

With these evolution and natural selection based algorithms what you hope for are **solutions that can adapt to changing environments**. These can be very useful for the practical use of computational algorithms in real life; for example in self driving cars or protein engineering.


## Introduction
Genetic algorithms use random exploration of the problem space combined with evolutionary processes like mutations and crossovers (exchange of genetic information) to improve guesses. But also, because they have no experience in the problem domain, they could try things a human would never think to try.

### Genetic algorithms (generally) have the following features: 

1. Population of chromosomes
2. Selection according to fitness
3. Crossovers to produce new offsprings
4. Random mutations of new offsprings

It's important to understand that chromosomes can essentuially represent anything. For all intents and purposes they're just a data strucuture with some unique properties. The example we're working with will perfectly illustrate this.

### A simple genetic algorithm:

1. Start with a randomly generated population
2. Calculate fitness of each member of the population
3. Mutate and crossover to produce **n** off springs
4. Replace current population with new population
5. Go to step 2

### Terminilogy
Before we proceed, let's get some definitions out of the way so we're all on the same page. 

**Population:** this is your dataset. You can think of each individual in the population as a possible solution to the problem. But let's just say the solution we want is not in there, so we're going to "evolve" this population using different strategies and some randomness to create new individuals that could be a better solution to the problem. 

For our example think of the dataset as a list of random strings like: "hcdsicdscdsc", "cdnsc cdscds", "blue world", "what this", "nonsense". None of these are what we want (aka; not our ideal solution), but you can see how we could potentially translate a string like blue world into hello world by replacing letters, changing order or randomly adding extra letters (these are called mutations).

**Chromosome:** A chromosome contains may genes. Each gene encodes a certain characteristic. Biologically, a gene can either give you blue eyes or brown eyes, or they could mutate and end up giving you green eyes. It's more complicated than that, but you get the gist.  

**Mutations:** For this specific case we will define mutations as replacing one letter with another in a given location of the string. This mimics what actual mutations do, where they replace a base in a given gene of your chromosome. 

### The code

In [1]:
from random import (choice, random, randint)

TARGET_CHROMOSOME = "Hello, world!"

#### Chromosome
For simplicity let's define our population as a set of chromosomes, each consisting of 13 genes each containing one Unicode character. You can think a chromosome to be aspiring to be the string "Hello, world!" by making sure each of it's genes have the right character at their respective positions. This would be what we'll be looking for when we mate and mutate our population.

The following code defines the chromosome. It should be able to perform mutations and crossovers on its own genes. Additionally we're also defining a method that will calculate the fitness of the chromosome (which in this case also represents the fitness of the individual), and a method for randomly generating chromosome (we will use this to generate our initial population.)

In [2]:
class Chromosome:
#   Initialize the chromosome with a gene (some string of characters) and calculate the fitness of the 
#   chromosome based on the fitness function that we wrote. 
    def __init__(self, genes):
        self.genes = genes
        self.fitness = Chromosome._update_fitness(genes)
    
#   Mates two individuals in the population. It picks a position 
#   in each of the chromosome's genes and exhanges them with each other. 
    def mate(self, mate):
        pivot = randint(0, len(self.genes) - 1)
        genes1 = self.genes[:pivot] + mate.genes[pivot:]
        genes2 = mate.genes[:pivot] + self.genes[pivot:]
        
        return Chromosome(genes1), Chromosome(genes2)
    
#   Mutates a chromosome by picking a random gene and changing its Unicode character
    def mutate(self):
        genes = list(self.genes)
        delta = randint(32, 121)
        idx = randint(0, len(genes) - 1)
        genes[idx] = chr((ord(genes[idx]) + delta) % 122)
        
        return Chromosome(''.join(genes))

#   Calculates and updates the fitness of a chromosome. All Unicode characters have a 
#   numeric representation. We can therefor use the total of the difference between the characters 
#   at each position of the chromosome to calcualte a score that represents how far a certain chromosome
#   differs from the target chromosome. 
    @staticmethod            
    def _update_fitness(genes):
        fitness = 0
#       Zip iterates through genes and target chromosome at the same time
        for a, b in zip(genes, TARGET_CHROMOSOME):
            fitness += abs(ord(a) - ord(b))
            
        return fitness
        
#   Takes the target chromosome and randomly replaces the character at each position to 
#   generate new random chromosomes.
    @staticmethod
    def gen_random():
        genes = []
        for x in range(len(TARGET_CHROMOSOME)):
            genes.append(chr(randint(32, 121)))
        
        return Chromosome(''.join(genes))

#### Population
This class will be what can generate a population of chromosomes (== individuals) that have a some properties like a `probability of mutating` and `probablity of crossovers`. Both of these will be used when a population is "evolved" by mating the current population to produce a new population.

In [3]:
class Population:
    _tournamentSize = 3
    
    def __init__(self, size=1024, crossover=0.8, mutation=0.03):
        self.mutation = mutation
        self.crossover = crossover
        
        buf = []
        for i in range(size): buf.append(Chromosome.gen_random())
        self.population = list(sorted(buf, key=lambda x: x.fitness))
    
#   We will be using a tournament selection algorithm for selecting parents for 
#   mating in each generation. It would simply use our fitness method to find the fittest 
#   individuals in the population to mate. There are various other selection algorithms you 
#   could use, but this method works just fine for this example. 
    def _tournament_selection(self):
        best = choice(self.population)
        for i in range(Population._tournamentSize):
            cont = choice(self.population)
            if (cont.fitness < best.fitness): best = cont
                    
        return best


    def _selectParents(self):            
        return (self._tournament_selection(), self._tournament_selection())
     
#   A pair of parents are selected, with probability Pc they cross over to form two offspring. 
#   If they do not cross over, then the offspring are exact copies of each parent. 
#   After that each off spring is subjected to mutations based on Pm.
    def evolve(self):
        size = len(self.population)
        idx = int(round(size * 0.2))
        buf = self.population[:idx]
        
        while (idx < size):
            if random() <= self.crossover:
                (p1, p2) = self._selectParents()
                children = p1.mate(p2)
                for c in children:
                    if random() <= self.mutation:
                        buf.append(c.mutate())
                    else:
                        buf.append(c)
                idx += 2
            else:
                if random() <= self.mutation:
                    buf.append(self.population[idx].mutate())
                else:
                    buf.append(self.population[idx])
                idx += 1
        
        self.population = list(sorted(buf[:size], key=lambda x: x.fitness))

### Running the code

In [4]:
maxGenerations = 20_000
initial_population = Population(size=2048, crossover=0.8, mutation=0.3)

for i in range(1, maxGenerations + 1):
    print("Generation %d: %s" % (i, initial_population.population[0].genes))
    if initial_population.population[0].fitness == 0: break
    else: initial_population.evolve()
else:
    print("Cannot complete.")

Generation 1: CvroOM`ulyb_ 
Generation 2: Sbihs,Sxbwd/)
Generation 3: QHrTiI!byyjS 
Generation 4: GIihs,SxblYh1
Generation 5: Sbihs,`ulyb_ 
Generation 6: Namvr!)skkq]
Generation 7: Namvr!)skkq]
Generation 8: Sbihs,xmgpU 
Generation 9: Bakor!-wqurg!
Generation 10: Sbihs,xmkqg!
Generation 11: Sbihs,xmkqg!
Generation 12: Nbihs.!wpqdi 
Generation 13: Jeihs.!wpqdi 
Generation 14: Jeihs.!wpqdi 
Generation 15: Jeihs.!wpqcd 
Generation 16: Jeihs.!wpqcd 
Generation 17: Jbihn.!wpqod 
Generation 18: Jbihn.!wpqod 
Generation 19: Jbpkm.!wpqod 
Generation 20: Jbpkm.!wpqod 
Generation 21: Jeihn.!wpqod 
Generation 22: Gamln.!wpqod 
Generation 23: Jemks.!wpqld 
Generation 24: Jemks.!wpqld!
Generation 25: Jemks,!wpqld 
Generation 26: Ibkkm,!wpqld!
Generation 27: Ibkkm,!wpqld!
Generation 28: Gekom,!wpqld!
Generation 29: Jemkm,!wpqld!
Generation 30: Gemkm,!wpqld!
Generation 31: Gemkm,!wpqld!
Generation 32: Iemln,!wpqld 
Generation 33: Iemln,!wpqld 
Generation 34: Iemln,!wpqld 
Generation 35: Iemln,!w

Generation 292: Helln, woqld!
Generation 293: Helln, woqld!
Generation 294: Helln, woqld!
Generation 295: Helln, woqld!
Generation 296: Helln, woqld!
Generation 297: Helln, woqld!
Generation 298: Helln, woqld!
Generation 299: Helln, woqld!
Generation 300: Helln, woqld!
Generation 301: Helln, woqld!
Generation 302: Helln, woqld!
Generation 303: Helln, woqld!
Generation 304: Helln, woqld!
Generation 305: Helln, woqld!
Generation 306: Helln, woqld!
Generation 307: Helln, woqld!
Generation 308: Helln, woqld!
Generation 309: Helln, woqld!
Generation 310: Helln, woqld!
Generation 311: Helln, woqld!
Generation 312: Helln, woqld!
Generation 313: Helln, woqld!
Generation 314: Helln, woqld!
Generation 315: Helln, woqld!
Generation 316: Helln, woqld!
Generation 317: Helln, woqld!
Generation 318: Helln, woqld!
Generation 319: Helln, woqld!
Generation 320: Helln, woqld!
Generation 321: Helln, woqld!
Generation 322: Helln, woqld!
Generation 323: Helln, woqld!
Generation 324: Helln, woqld!
Generation

Generation 580: Helln, woqld!
Generation 581: Helln, woqld!
Generation 582: Helln, woqld!
Generation 583: Helln, woqld!
Generation 584: Helln, woqld!
Generation 585: Helln, woqld!
Generation 586: Helln, woqld!
Generation 587: Helln, woqld!
Generation 588: Helln, woqld!
Generation 589: Helln, woqld!
Generation 590: Helln, woqld!
Generation 591: Helln, woqld!
Generation 592: Helln, woqld!
Generation 593: Helln, woqld!
Generation 594: Helln, woqld!
Generation 595: Helln, woqld!
Generation 596: Helln, woqld!
Generation 597: Helln, woqld!
Generation 598: Helln, woqld!
Generation 599: Helln, woqld!
Generation 600: Helln, woqld!
Generation 601: Helln, woqld!
Generation 602: Helln, woqld!
Generation 603: Helln, woqld!
Generation 604: Helln, woqld!
Generation 605: Helln, woqld!
Generation 606: Helln, woqld!
Generation 607: Helln, woqld!
Generation 608: Helln, woqld!
Generation 609: Helln, woqld!
Generation 610: Helln, woqld!
Generation 611: Helln, woqld!
Generation 612: Helln, woqld!
Generation

Generation 864: Helln, world!
Generation 865: Helln, world!
Generation 866: Helln, world!
Generation 867: Helln, world!
Generation 868: Helln, world!
Generation 869: Helln, world!
Generation 870: Helln, world!
Generation 871: Helln, world!
Generation 872: Helln, world!
Generation 873: Helln, world!
Generation 874: Helln, world!
Generation 875: Helln, world!
Generation 876: Helln, world!
Generation 877: Helln, world!
Generation 878: Helln, world!
Generation 879: Helln, world!
Generation 880: Helln, world!
Generation 881: Helln, world!
Generation 882: Helln, world!
Generation 883: Helln, world!
Generation 884: Helln, world!
Generation 885: Helln, world!
Generation 886: Helln, world!
Generation 887: Helln, world!
Generation 888: Helln, world!
Generation 889: Helln, world!
Generation 890: Helln, world!
Generation 891: Helln, world!
Generation 892: Helln, world!
Generation 893: Helln, world!
Generation 894: Helln, world!
Generation 895: Helln, world!
Generation 896: Helln, world!
Generation

Generation 1138: Helln, world!
Generation 1139: Helln, world!
Generation 1140: Helln, world!
Generation 1141: Helln, world!
Generation 1142: Helln, world!
Generation 1143: Helln, world!
Generation 1144: Helln, world!
Generation 1145: Helln, world!
Generation 1146: Helln, world!
Generation 1147: Helln, world!
Generation 1148: Helln, world!
Generation 1149: Helln, world!
Generation 1150: Helln, world!
Generation 1151: Helln, world!
Generation 1152: Helln, world!
Generation 1153: Helln, world!
Generation 1154: Helln, world!
Generation 1155: Helln, world!
Generation 1156: Helln, world!
Generation 1157: Helln, world!
Generation 1158: Helln, world!
Generation 1159: Helln, world!
Generation 1160: Helln, world!
Generation 1161: Helln, world!
Generation 1162: Helln, world!
Generation 1163: Helln, world!
Generation 1164: Helln, world!
Generation 1165: Helln, world!
Generation 1166: Helln, world!
Generation 1167: Helln, world!
Generation 1168: Helln, world!
Generation 1169: Helln, world!
Generati

...

Generation 4682: Helln, world!
Generation 4683: Helln, world!
Generation 4684: Helln, world!
Generation 4685: Helln, world!
Generation 4686: Helln, world!
Generation 4687: Helln, world!
Generation 4688: Helln, world!
Generation 4689: Helln, world!
Generation 4690: Helln, world!
Generation 4691: Helln, world!
Generation 4692: Helln, world!
Generation 4693: Helln, world!
Generation 4694: Helln, world!
Generation 4695: Helln, world!
Generation 4696: Helln, world!
Generation 4697: Helln, world!
Generation 4698: Helln, world!
Generation 4699: Helln, world!
Generation 4700: Helln, world!
Generation 4701: Helln, world!
Generation 4702: Helln, world!
Generation 4703: Helln, world!
Generation 4704: Helln, world!
Generation 4705: Helln, world!
Generation 4706: Helln, world!
Generation 4707: Helln, world!
Generation 4708: Helln, world!
Generation 4709: Helln, world!
Generation 4710: Helln, world!
Generation 4711: Helln, world!
Generation 4712: Helln, world!
Generation 4713: Helln, world!
Generati

Generation 4962: Helln, world!
Generation 4963: Helln, world!
Generation 4964: Helln, world!
Generation 4965: Helln, world!
Generation 4966: Helln, world!
Generation 4967: Helln, world!
Generation 4968: Helln, world!
Generation 4969: Helln, world!
Generation 4970: Helln, world!
Generation 4971: Helln, world!
Generation 4972: Helln, world!
Generation 4973: Helln, world!
Generation 4974: Helln, world!
Generation 4975: Helln, world!
Generation 4976: Helln, world!
Generation 4977: Helln, world!
Generation 4978: Helln, world!
Generation 4979: Helln, world!
Generation 4980: Helln, world!
Generation 4981: Helln, world!
Generation 4982: Helln, world!
Generation 4983: Helln, world!
Generation 4984: Helln, world!
Generation 4985: Helln, world!
Generation 4986: Helln, world!
Generation 4987: Helln, world!
Generation 4988: Helln, world!
Generation 4989: Helln, world!
Generation 4990: Helln, world!
Generation 4991: Helln, world!
Generation 4992: Helln, world!
Generation 4993: Helln, world!
Generati

Generation 5238: Helln, world!
Generation 5239: Helln, world!
Generation 5240: Helln, world!
Generation 5241: Helln, world!
Generation 5242: Helln, world!
Generation 5243: Helln, world!
Generation 5244: Helln, world!
Generation 5245: Helln, world!
Generation 5246: Helln, world!
Generation 5247: Helln, world!
Generation 5248: Helln, world!
Generation 5249: Helln, world!
Generation 5250: Helln, world!
Generation 5251: Helln, world!
Generation 5252: Helln, world!
Generation 5253: Helln, world!
Generation 5254: Helln, world!
Generation 5255: Helln, world!
Generation 5256: Helln, world!
Generation 5257: Helln, world!
Generation 5258: Helln, world!
Generation 5259: Helln, world!
Generation 5260: Helln, world!
Generation 5261: Helln, world!
Generation 5262: Helln, world!
Generation 5263: Helln, world!
Generation 5264: Helln, world!
Generation 5265: Helln, world!
Generation 5266: Helln, world!
Generation 5267: Helln, world!
Generation 5268: Helln, world!
Generation 5269: Helln, world!
Generati

Generation 5509: Helln, world!
Generation 5510: Helln, world!
Generation 5511: Helln, world!
Generation 5512: Helln, world!
Generation 5513: Helln, world!
Generation 5514: Helln, world!
Generation 5515: Helln, world!
Generation 5516: Helln, world!
Generation 5517: Helln, world!
Generation 5518: Helln, world!
Generation 5519: Helln, world!
Generation 5520: Helln, world!
Generation 5521: Helln, world!
Generation 5522: Helln, world!
Generation 5523: Helln, world!
Generation 5524: Helln, world!
Generation 5525: Helln, world!
Generation 5526: Helln, world!
Generation 5527: Helln, world!
Generation 5528: Helln, world!
Generation 5529: Helln, world!
Generation 5530: Helln, world!
Generation 5531: Helln, world!
Generation 5532: Helln, world!
Generation 5533: Helln, world!
Generation 5534: Helln, world!
Generation 5535: Helln, world!
Generation 5536: Helln, world!
Generation 5537: Helln, world!
Generation 5538: Helln, world!
Generation 5539: Helln, world!
Generation 5540: Helln, world!
Generati

Generation 5788: Helln, world!
Generation 5789: Helln, world!
Generation 5790: Helln, world!
Generation 5791: Helln, world!
Generation 5792: Helln, world!
Generation 5793: Helln, world!
Generation 5794: Helln, world!
Generation 5795: Helln, world!
Generation 5796: Helln, world!
Generation 5797: Helln, world!
Generation 5798: Helln, world!
Generation 5799: Helln, world!
Generation 5800: Helln, world!
Generation 5801: Helln, world!
Generation 5802: Helln, world!
Generation 5803: Helln, world!
Generation 5804: Helln, world!
Generation 5805: Helln, world!
Generation 5806: Helln, world!
Generation 5807: Helln, world!
Generation 5808: Helln, world!
Generation 5809: Helln, world!
Generation 5810: Helln, world!
Generation 5811: Helln, world!
Generation 5812: Helln, world!
Generation 5813: Helln, world!
Generation 5814: Helln, world!
Generation 5815: Helln, world!
Generation 5816: Helln, world!
Generation 5817: Helln, world!
Generation 5818: Helln, world!
Generation 5819: Helln, world!
Generati